# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon import Lexicon_FR_updated as lexicon_fr
from utils.io import boreholes_from_files, striplog_from_text,gdf_viewer,\
df_from_sources, striplog_from_df
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
df_all = pd.read_csv('test_data_boreholes.csv', sep=',')

## Data Checking

In [3]:
df_all.drop('Z', axis='columns', inplace=True)
df_all.rename({'new_Z':'Z'}, axis='columns', inplace=True)

## Create Boreholes from dataframes or files

In [4]:
df_all[:10]

,ID,X,Y,Z,Diameter,Length,Lithology,Colour,Thickness,Top_litho,Base_litho
0,BH001,152248.0,121422.0,102.0,0.46,37,sand,olivedrab,4,0,4
1,BH001,152248.0,121422.0,98.0,0.46,37,clay,slateblue,1,4,5
2,BH001,152248.0,121422.0,97.0,0.46,37,sandstone,blue,10,5,15
3,BH001,152248.0,121422.0,87.0,0.46,37,coal,snow,2,15,17
4,BH001,152248.0,121422.0,85.0,0.46,37,limestone,darkslategrey,20,17,37
5,BH002,152669.0,121128.0,100.0,0.33,42,sand,olivedrab,6,0,6
6,BH002,152669.0,121128.0,94.0,0.33,42,clay,slateblue,3,6,9
7,BH002,152669.0,121128.0,91.0,0.33,42,sandstone,blue,9,9,18
8,BH002,152669.0,121128.0,82.0,0.33,42,coal,snow,7,18,25
9,BH002,152669.0,121128.0,75.0,0.33,42,limestone,darkslategrey,17,25,42


In [5]:
bhs_df=[df_all]

In [6]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Lithology',
                                                lexicon=Lexicon.default(),
                                            diam_field='Diameter', 
                                            thick_field='Thickness')


Dataframe 0 processing...
|__ID:'BH001'
Error : No lithology matching with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=37.0)]
{0: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f40c4408bb0>, 'base': <core.orm.PositionOrm object at 0x7f4094cb17f0>}, 1: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f4094c45e20>, 'base': <core.orm.PositionOrm object at 0x7f4094c45df0>}, 2: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f4094c45250>, 'base': <core.orm.PositionOrm object at 0x7f4094c45be0>}, 3: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f4094c45f10>, 'base': <core.orm.PositionOrm object at 0x7f4094c45f40>}, 4: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f4094c45d30>, 'base': <core.orm.PositionOrm object at 0x7f4094c5c4c0>}}

|__ID:'BH00

/home/yanathan/Projects/GSDMA/utils/io.py:505: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=id_))
/home/yanathan/Projects/GSDMA/utils/io.py:505: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to Lin


|__ID:'BH033'
Error : No lithology matching with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=39.0)]
{160: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f4094b3b580>, 'base': <core.orm.PositionOrm object at 0x7f4094b3bd60>}, 161: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f4094b3be50>, 'base': <core.orm.PositionOrm object at 0x7f4094b33160>}, 162: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f4094b33040>, 'base': <core.orm.PositionOrm object at 0x7f4094b33730>}, 163: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f4094b33fd0>, 'base': <core.orm.PositionOrm object at 0x7f4094b33c40>}, 164: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f4094b33b20>, 'base': <core.orm.PositionOrm object at 0x7f4094b33490>}}

|__ID:'BH034'
Error : No li

|__ID:'BH070'
Error : No lithology matching with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=43.0)]
{345: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f4094a4cc70>, 'base': <core.orm.PositionOrm object at 0x7f4094a4cb50>}, 346: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f4094a4ca30>, 'base': <core.orm.PositionOrm object at 0x7f4094a4cb20>}, 347: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f4094a4ca60>, 'base': <core.orm.PositionOrm object at 0x7f4094a4cca0>}, 348: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f4094a4cdc0>, 'base': <core.orm.PositionOrm object at 0x7f4094a4cdf0>}, 349: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f4094a4ccd0>, 'base': <core.orm.PositionOrm object at 0x7f4094a4cee0>}}

|__ID:'BH071'
Error : No lit

|__ID:'BH121'
Error : No lithology matching with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=23.0)]
{600: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f4094903490>, 'base': <core.orm.PositionOrm object at 0x7f4094903460>}, 601: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f40949223d0>, 'base': <core.orm.PositionOrm object at 0x7f4094917460>}, 602: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f4094917d00>, 'base': <core.orm.PositionOrm object at 0x7f4094917c70>}, 603: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f40949179a0>, 'base': <core.orm.PositionOrm object at 0x7f4094917bb0>}, 604: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f4094917d30>, 'base': <core.orm.PositionOrm object at 0x7f4094917820>}}

|__ID:'BH122'
Error : No lit

|__ID:'BH165'
Error : No lithology matching with 'coal' in given lexicon
Summary : [Striplog(5 Intervals, start=0.0, stop=19.0)]
{820: {'description': 'sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f409482ebe0>, 'base': <core.orm.PositionOrm object at 0x7f4094bf1d00>}, 821: {'description': 'clay', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f4094bf1910>, 'base': <core.orm.PositionOrm object at 0x7f4094bfb2e0>}, 822: {'description': 'sandstone', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f4094bfb9d0>, 'base': <core.orm.PositionOrm object at 0x7f4094bfb4c0>}, 823: {'description': 'white sand', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f4094bfb640>, 'base': <core.orm.PositionOrm object at 0x7f4094bfb790>}, 824: {'description': 'limestone', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7f4094bfb6d0>, 'base': <core.orm.PositionOrm object at 0x7f4094bfb550>}}

|__ID:'BH166'
Error : No lit

In [7]:
boreholes

[<core.orm.BoreholeOrm>(Name=BH001, Length=37, Diameter=0.46, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH002, Length=42, Diameter=0.33, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH003, Length=27, Diameter=0.42, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH004, Length=27, Diameter=0.66, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH005, Length=24, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH006, Length=19, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH007, Length=29, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH008, Length=45, Diameter=0.62, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH009, Length=24, Diameter=0.56, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH10, Length=32, Diameter=0.35, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH011, Length=35, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH012, Length=19, Diameter=0.4, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH013, Length=39, Diameter=0.54, Intervals=5),
 <core.orm.Bor

In [8]:
components

{0: Component({'lithology': 'limestone'}),
 1: Component({'lithology': 'sandstone'}),
 2: Component({'lithology': 'sand'}),
 3: Component({'lithology': 'sand', 'colour': 'white'}),
 4: Component({'lithology': 'clay'})}

In [9]:
len(boreholes), len(components)

(199, 5)

## Display the striplog and 3D view

In [10]:
remove('tmp_files/tfe_orm_db.db')

In [11]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [12]:
Base.metadata.create_all(engine)

2021-06-08 00:03:33,869 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-08 00:03:33,870 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-06-08 00:03:33,870 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-08 00:03:33,871 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-06-08 00:03:33,872 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-08 00:03:33,873 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-06-08 00:03:33,873 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-08 00:03:33,874 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-06-08 00:03:33,875 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-08 00:03:33,876 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-06-08 00:03:33,877 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-08 00:03:33,878 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-06-08 00:03:33,878 INFO sqlalchemy.engine.Engine 

In [13]:
Session = sessionmaker(bind=engine)
session = Session()

In [14]:
p = Project(session)
p.add_components(components)

2021-06-08 00:03:33,921 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-08 00:03:33,925 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-08 00:03:33,926 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()
2021-06-08 00:03:33,929 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-06-08 00:03:33,930 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ((0, 'Limestone'), (1, 'Sandstone'), (2, 'Sand'), (3, 'Sand, white'), (4, 'Clay'))
2021-06-08 00:03:33,931 INFO sqlalchemy.engine.Engine COMMIT
2021-06-08 00:03:33,934 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-08 00:03:33,935 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-08 00:03:33,935 INFO sqlalchemy.engi

Add boreholes into the project

In [15]:
for bh in boreholes:
    p.add_borehole(bh)

2021-06-08 00:03:33,944 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-08 00:03:33,946 INFO sqlalchemy.engine.Engine [generated in 0.00210s] ('BH001', 37.0, 0.46)
2021-06-08 00:03:33,948 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-08 00:03:33,948 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ((0, 0.0, 0.0, 0.0, 152248.0, 121422.0), (1, 4.0, 4.0, 4.0, 152248.0, 121422.0), (2, 4.0, 4.0, 4.0, 152248.0, 121422.0), (3, 5.0, 5.0, 5.0, 152248.0, 121422.0), (4, 5.0, 5.0, 5.0, 152248.0, 121422.0), (5, 15.0, 15.0, 15.0, 152248.0, 121422.0), (6, 15.0, 15.0, 15.0, 152248.0, 121422.0), (7, 17.0, 17.0, 17.0, 152248.0, 121422.0), (8, 17.0, 17.0, 17.0, 152248.0, 121422.0), (9, 37.0, 37.0, 37.0, 152248.0, 121422.0))
2021-06-08 00:03:33,951 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, 

2021-06-08 00:03:34,057 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:34,058 INFO sqlalchemy.engine.Engine [cached since 0.09096s ago] ('BH002',)
2021-06-08 00:03:34,060 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:34,061 INFO sqlalchemy.engine.Engine [cached since 0.08763s ago] (10,)
2021-06-08 00:03:34,063 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".

2021-06-08 00:03:34,152 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:34,153 INFO sqlalchemy.engine.Engine [cached since 0.1755s ago] (23,)
2021-06-08 00:03:34,155 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:34,156 INFO sqlalchemy.engine.Engine [cached since 0.1822s ago] (24,)
2021-06-08 00:03:34,157 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "

2021-06-08 00:03:34,241 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:34,242 INFO sqlalchemy.engine.Engine [cached since 0.2652s ago] (37,)
2021-06-08 00:03:34,244 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:34,245 INFO sqlalchemy.engine.Engine [cached since 0.2712s ago] (38,)
2021-06-08 00:03:34,246 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "

2021-06-08 00:03:34,368 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-08 00:03:34,369 INFO sqlalchemy.engine.Engine [cached since 0.4211s ago] ((50, 0.0, 0.0, 0.0, 152669.0, 121128.0), (51, 1.0, 1.0, 1.0, 152669.0, 121128.0), (52, 1.0, 1.0, 1.0, 152669.0, 121128.0), (53, 2.0, 2.0, 2.0, 152669.0, 121128.0), (54, 2.0, 2.0, 2.0, 152669.0, 121128.0), (55, 7.0, 7.0, 7.0, 152669.0, 121128.0), (56, 7.0, 7.0, 7.0, 152669.0, 121128.0), (57, 12.0, 12.0, 12.0, 152669.0, 121128.0), (58, 12.0, 12.0, 12.0, 152669.0, 121128.0), (59, 19.0, 19.0, 19.0, 152669.0, 121128.0))
2021-06-08 00:03:34,370 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-06-08 00:03:34,371 INFO sqlalchemy.engine.Engine [cached since 0.4198s ago] ((25, 'BH006', 0, 'sand', 50, 51), (26, 'BH006', 1, 'clay', 52, 53), (27, 'BH006', 2, 'sandstone', 54, 55), (28, 'B

...

2021-06-08 00:03:49,952 INFO sqlalchemy.engine.Engine [cached since 15.98s ago] (1938,)
2021-06-08 00:03:49,953 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:49,954 INFO sqlalchemy.engine.Engine [cached since 15.98s ago] (1939,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
Borehole geometry created successfully !
2021-06-08 00:03:49,998 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-08 00:03:49,999 INFO sqlalchemy.engine.Engine [cached since 16.06s ago] ('BH195', 30.0, 0.65)
2021-06-08 00:03:50,000 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?,

2021-06-08 00:03:50,086 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-06-08 00:03:50,086 INFO sqlalchemy.engine.Engine [cached since 16.14s ago] ((975, 'BH196', 0, 'sand', 1950, 1951), (976, 'BH196', 1, 'clay', 1952, 1953), (977, 'BH196', 2, 'sandstone', 1954, 1955), (978, 'BH196', 3, 'white sand', 1956, 1957), (979, 'BH196', 4, 'limestone', 1958, 1959))
2021-06-08 00:03:50,087 INFO sqlalchemy.engine.Engine COMMIT
2021-06-08 00:03:50,093 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-08 00:03:50,094 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-08 00:03:50,095 INFO sqlalchemy.engine.Engine [cached since 16.17s ago] ()
2021-06-08 00:03:50,100 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Interva

2021-06-08 00:03:50,200 INFO sqlalchemy.engine.Engine [cached since 16.23s ago] (1960,)
2021-06-08 00:03:50,201 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:50,202 INFO sqlalchemy.engine.Engine [cached since 16.22s ago] (1961,)
2021-06-08 00:03:50,203 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:50,204 INFO sqlalchemy.engine.Engine [cached since 16.23s ago] (1962,)
2021-06-08 00:03:50,205 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Position

2021-06-08 00:03:50,286 INFO sqlalchemy.engine.Engine [cached since 16.31s ago] (1974,)
2021-06-08 00:03:50,288 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:50,288 INFO sqlalchemy.engine.Engine [cached since 16.31s ago] (1975,)
2021-06-08 00:03:50,290 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:50,290 INFO sqlalchemy.engine.Engine [cached since 16.32s ago] (1976,)
2021-06-08 00:03:50,292 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Position

2021-06-08 00:03:50,373 INFO sqlalchemy.engine.Engine [cached since 16.4s ago] (1988,)
2021-06-08 00:03:50,375 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-06-08 00:03:50,375 INFO sqlalchemy.engine.Engine [cached since 16.4s ago] (1989,)
No given legend or incorrect format ! A default legend will be used
Borehole geometry created successfully !
Borehole geometry created successfully !


In [16]:
p.add_link_between_components_and_intervals(link_intv_comp)

2021-06-08 00:03:50,452 INFO sqlalchemy.engine.Engine INSERT INTO "Linkintervalcomponent" (int_id, comp_id, extra_data) VALUES (?, ?, ?)
2021-06-08 00:03:50,453 INFO sqlalchemy.engine.Engine [generated in 0.00250s] ((0, 2, ''), (1, 4, ''), (2, 1, ''), (3, 3, ''), (4, 0, ''), (5, 2, ''), (6, 4, ''), (7, 1, '')  ... displaying 10 of 995 total bound parameter sets ...  (993, 3, ''), (994, 0, ''))
2021-06-08 00:03:50,463 INFO sqlalchemy.engine.Engine COMMIT
2021-06-08 00:03:50,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-08 00:03:50,469 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-08 00:03:50,470 INFO sqlalchemy.engine.Engine [cached since 16.55s ago] ()


In [17]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

2021-06-08 00:03:50,478 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  199


In [18]:
p.boreholes

2021-06-08 00:03:50,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-08 00:03:50,488 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:50,489 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ('BH001',)
2021-06-08 00:03:50,491 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:50,492 INFO sqlalchemy.engine.Engine [cached since 16.52s ago] ('BH001',)
2021-06-08 00:03:50,494 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length"

2021-06-08 00:03:50,531 INFO sqlalchemy.engine.Engine [cached since 0.04331s ago] ('BH011',)
2021-06-08 00:03:50,533 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:50,534 INFO sqlalchemy.engine.Engine [cached since 16.57s ago] ('BH011',)
2021-06-08 00:03:50,536 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:50,537 INFO sqlalchemy.engine.Engine [cached since 0.04891s ago] ('BH012',)
2021-06-08 00:03:50,539 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".bo

2021-06-08 00:03:50,585 INFO sqlalchemy.engine.Engine [cached since 16.62s ago] ('BH021',)
2021-06-08 00:03:50,587 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:50,587 INFO sqlalchemy.engine.Engine [cached since 0.0996s ago] ('BH022',)
2021-06-08 00:03:50,589 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:50,589 INFO sqlalchemy.engine.Engine [cached since 16.62s ago] ('BH022',)
2021-06-08 00:03:50,591 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".lengt

2021-06-08 00:03:50,634 INFO sqlalchemy.engine.Engine [cached since 0.1463s ago] ('BH032',)
2021-06-08 00:03:50,635 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:50,636 INFO sqlalchemy.engine.Engine [cached since 16.67s ago] ('BH032',)
2021-06-08 00:03:50,637 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:50,638 INFO sqlalchemy.engine.Engine [cached since 0.1502s ago] ('BH033',)
2021-06-08 00:03:50,639 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".bore

2021-06-08 00:03:50,683 INFO sqlalchemy.engine.Engine [cached since 16.72s ago] ('BH042',)
2021-06-08 00:03:50,684 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:50,685 INFO sqlalchemy.engine.Engine [cached since 0.1973s ago] ('BH043',)
2021-06-08 00:03:50,687 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:50,687 INFO sqlalchemy.engine.Engine [cached since 16.72s ago] ('BH043',)
2021-06-08 00:03:50,689 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".lengt

...

2021-06-08 00:03:51,244 INFO sqlalchemy.engine.Engine [cached since 0.7558s ago] ('BH158',)
2021-06-08 00:03:51,245 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:51,245 INFO sqlalchemy.engine.Engine [cached since 17.28s ago] ('BH158',)
2021-06-08 00:03:51,247 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:51,248 INFO sqlalchemy.engine.Engine [cached since 0.7599s ago] ('BH159',)
2021-06-08 00:03:51,249 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".bore

2021-06-08 00:03:51,292 INFO sqlalchemy.engine.Engine [cached since 17.32s ago] ('BH168',)
2021-06-08 00:03:51,293 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:51,294 INFO sqlalchemy.engine.Engine [cached since 0.806s ago] ('BH169',)
2021-06-08 00:03:51,295 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:51,296 INFO sqlalchemy.engine.Engine [cached since 17.33s ago] ('BH169',)
2021-06-08 00:03:51,297 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length

2021-06-08 00:03:51,341 INFO sqlalchemy.engine.Engine [cached since 0.8529s ago] ('BH179',)
2021-06-08 00:03:51,342 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:51,343 INFO sqlalchemy.engine.Engine [cached since 17.38s ago] ('BH179',)
2021-06-08 00:03:51,344 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:51,345 INFO sqlalchemy.engine.Engine [cached since 0.8571s ago] ('BH180',)
2021-06-08 00:03:51,346 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".bore

2021-06-08 00:03:51,386 INFO sqlalchemy.engine.Engine [cached since 17.42s ago] ('BH189',)
2021-06-08 00:03:51,388 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2021-06-08 00:03:51,389 INFO sqlalchemy.engine.Engine [cached since 0.9012s ago] ('BH190',)
2021-06-08 00:03:51,391 INFO sqlalchemy.engine.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2021-06-08 00:03:51,392 INFO sqlalchemy.engine.Engine [cached since 17.42s ago] ('BH190',)
2021-06-08 00:03:51,394 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".lengt

[<core.orm.BoreholeOrm>(Name=BH001, Length=37.0, Diameter=0.46, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH002, Length=42.0, Diameter=0.33, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH003, Length=27.0, Diameter=0.42, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH004, Length=27.0, Diameter=0.66, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH005, Length=24.0, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH006, Length=19.0, Diameter=0.31, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH007, Length=29.0, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH008, Length=45.0, Diameter=0.62, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH009, Length=24.0, Diameter=0.56, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH10, Length=32.0, Diameter=0.35, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH011, Length=35.0, Diameter=0.58, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH012, Length=19.0, Diameter=0.4, Intervals=5),
 <core.orm.BoreholeOrm>(Name=BH013, Length=39.0, Diameter=0.54, In

In [19]:
p.refresh(verbose=True)

2021-06-08 00:03:51,448 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-08 00:03:51,449 INFO sqlalchemy.engine.Engine [cached since 17.52s ago] ()


In [20]:
session.close()

2021-06-08 00:03:51,457 INFO sqlalchemy.engine.Engine ROLLBACK


## Display one object

In [21]:
p.boreholes_3d[0].components

[Component({'lithology': 'limestone'}),
 Component({'lithology': 'sandstone'}),
 Component({'lithology': 'sand'}),
 Component({'lithology': 'sand', 'colour': 'white'}),
 Component({'lithology': 'clay'})]

In [22]:
i = 'BH001'
df_all.query(f"ID=='{i}'")#['Thickness'].cumsum().max()

,ID,X,Y,Z,Diameter,Length,Lithology,Colour,Thickness,Top_litho,Base_litho
0,BH001,152248.0,121422.0,102.0,0.46,37,sand,olivedrab,4,0,4
1,BH001,152248.0,121422.0,98.0,0.46,37,clay,slateblue,1,4,5
2,BH001,152248.0,121422.0,97.0,0.46,37,sandstone,blue,10,5,15
3,BH001,152248.0,121422.0,87.0,0.46,37,coal,snow,2,15,17
4,BH001,152248.0,121422.0,85.0,0.46,37,limestone,darkslategrey,20,17,37


In [23]:
bh = p.boreholes_3d[0]
print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

BH001 
 [Interval({'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 4.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'sand', 'data': {}, 'components': [Component({'lithology': 'sand'})]}), Interval({'top': Position({'middle': 4.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 5.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'clay', 'data': {}, 'components': [Component({'lithology': 'clay'})]}), Interval({'top': Position({'middle': 5.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 15.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'sandstone', 'data': {}, 'components': [Component({'lithology': 'sandstone'})]}), Interval({'top': Position({'middle': 15.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'base': Position({'middle': 17.0, 'x': 152248.0, 'y': 121422.0, 'units': 'm'}), 'description': 'white

In [24]:
p.boreholes_3d[0].plot3d(x3d=True)

In [25]:
pause

NameError: name 'pause' is not defined

### Display project boreholes

### Boreholes in 3D using X3D for interaction within the notebook

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)